<a href="https://colab.research.google.com/github/pmopedro/computational-chess/blob/pedro/notebooks/Explore1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [70]:
import pandas as pd
import tensorflow as tf

In [71]:
datainfo =  "../data/chessData.csv" # proj_folder + 'chessData.csv'
df = pd.read_csv(datainfo)

# Exploratory Data Analysis

In [72]:
df.head()

,FEN,Evaluation
0,rnbqkbnr/pppppppp/8/8/4P3/8/PPPP1PPP/RNBQKBNR ...,-10
1,rnbqkbnr/pppp1ppp/4p3/8/4P3/8/PPPP1PPP/RNBQKBN...,+56
2,rnbqkbnr/pppp1ppp/4p3/8/3PP3/8/PPP2PPP/RNBQKBN...,-9
3,rnbqkbnr/ppp2ppp/4p3/3p4/3PP3/8/PPP2PPP/RNBQKB...,+52
4,rnbqkbnr/ppp2ppp/4p3/3p4/3PP3/8/PPPN1PPP/R1BQK...,-26


In [73]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12958035 entries, 0 to 12958034
Data columns (total 2 columns):
 #   Column      Dtype 
---  ------      ----- 
 0   FEN         object
 1   Evaluation  object
dtypes: object(2)
memory usage: 197.7+ MB


In [74]:
x_test=df['FEN'].tail(1000000)
print(x_test.shape)
y_test=df['Evaluation'].tail(1000000)
print(y_test.shape)
del df

(1000000,)
(1000000,)


In [75]:
import chess
def boardstate(fen):
    board = chess.Board(fen)
    fstr = str(fen)

    if board.has_kingside_castling_rights(chess.WHITE) == True:
        WCKI = 1
    else:
        WCKI = 0
    if board.has_queenside_castling_rights(chess.WHITE) == True:
        WCQ = 1
    else:
        WCQ = 0
    if board.is_check() == True:
        WCH = 1
    else:
        WCH = 0

    if board.has_kingside_castling_rights(chess.BLACK) == True:
        BCKI = 1
    else:
        BCKI = 0
    if board.has_queenside_castling_rights(chess.BLACK) == True:
        BCQ = 1
    else:
        BCQ = 0
    if board.was_into_check() == True:
        BCH = 1
    else:
        BCH = 0

    #f = [M, WCKI, WCQ, WCH, BCKI, BCQ, BCH]
    fw = [WCKI, WCQ, WCH]
    fb = [BCKI, BCQ, BCH]

    bstr = str(board)
    bstr = bstr.replace("p", "\ -1")
    bstr = bstr.replace("n", "\ -3")
    bstr = bstr.replace("b", "\ -4")
    bstr = bstr.replace("r", "\ -5")
    bstr = bstr.replace("q", "\ -9")
    bstr = bstr.replace("k", "\ -100")
    bstr = bstr.replace("P", "\ 1")
    bstr = bstr.replace("N", "\ 3")
    bstr = bstr.replace("B", "\ 4")
    bstr = bstr.replace("R", "\ 5")
    bstr = bstr.replace("Q", "\ 9")
    bstr = bstr.replace("K", "\ 100")
    bstr = bstr.replace(".", "\ 0")
    bstr = bstr.replace("\ ", ",")
    bstr = bstr.replace("'", " ")
    bstr = bstr.replace("\n", "")
    bstr = bstr.replace(" ", "")
    bstr = bstr[1:]
    bstr = eval(bstr)
    bstr = list(bstr)
    if "w" not in fstr:
        for i in range(len(bstr)):
            bstr[i] = bstr[i] * -1
        bstr.reverse()
        fs = fb
        fb = fw
        fw = fs


    BITBOARD = fw + fb + bstr


    return BITBOARD


In [76]:
import numpy as np
input2_columns = [0, 1, 2, 3, 4, 5]

data_features = x_test.apply(boardstate)
data_features = data_features.apply(pd.Series)
print(data_features.head())

          0   1   2   3   4   5   6    7    8   9   ...  60  61  62  63  64  \
11958035   0   0   0   0   0   0   0 -100    0   0  ...   1   0   5   0   0   
11958036   0   0   0   0   0   0   0 -100    0   0  ...   1   1   5   0   0   
11958037   0   0   0   0   0   0   0 -100    0   0  ...   1   0   5   0   0   
11958038   0   0   0   0   0   0   0    0 -100   0  ...   1   1   5   0   0   
11958039   0   0   0   0   0   0   0 -100    0   0  ...   1   0   5   0   0   

          65  66   67   68  69  
11958035   0   0    5  100   0  
11958036   0   0    0  100   0  
11958037   0   0    0  100   0  
11958038   0   0    0  100   0  
11958039   0   0  100    0   0  

[5 rows x 70 columns]


In [77]:
inputboard = data_features.drop(columns=data_features.iloc[:, input2_columns])
inputboard = np.array(inputboard)
inputmeta = data_features.iloc[:, input2_columns]
inputmeta = np.array(inputmeta)

In [78]:
def strfix(tr):
    if '#' in str(tr):
        if '-' in tr:
            t = -10000
        else:
            t = 10000
    elif '\ufeff+23' in str(tr):
        t = 0
    else:
        t = int(float(tr))

    t = t/10
    return t

y_test= y_test.apply(strfix)

In [79]:
import joblib
model_file1 = '../models/model_V100_7MM.pkl'

model1 = joblib.load(model_file1)

In [80]:
print("Evaluate on test data")
results = model1.predict([(inputboard), (inputmeta)])

Evaluate on test data
31250/31250 [==============================] - 369s 12ms/step


In [81]:
batch_size = 100  # Adjust the batch size according to your memory capacity
num_samples = x_test.shape[0]

mse_values = []
mae_values = []
for i in range(0, num_samples, batch_size):
    x_batch = results[i:i+batch_size]
    y_batch = y_test[i:i+batch_size]
    mse = tf.keras.losses.mean_squared_error(y_batch, x_batch).numpy()
    mae = tf.keras.losses.mean_absolute_error(y_batch, x_batch).numpy()
    mse_values.append(mse)
    mae_values.append(mae)

overall_mse = np.mean(mse_values)
overall_mae = np.mean(mae_values)
print("Overall Mean Squared Error:", overall_mse)
print("Overall Mean Absolute Error:", overall_mae)

Overall Mean Squared Error: 23919.574
Overall Mean Absolute Error: 55.818485


In [82]:
import os
import glob

def list_pickles(directory):
    """
    List all pickle files in a specified directory.

    Args:
    - directory: The path to the directory to search for pickle files.

    Returns:
    - List of paths to pickle files found in the directory.
    """
    pickle_files = []
    
    # Check if the directory exists
    if not os.path.exists(directory):
        print(f"Directory '{directory}' does not exist.")
        return pickle_files

    # Search for pickle files using glob
    search_pattern = os.path.join(directory, '*.pkl')
    pickle_files = glob.glob(search_pattern)

    return pickle_files


In [83]:
list_pickles("../models/")

['../models/model_7MM_DL2.pkl',
 '../models/model1.pkl',
 '../models/model.pkl',
 '../models/model_7MM_DL1.pkl',
 '../models/model_7MM_DL0.pkl',
 '../models/model_V100_7MM.pkl',
 '../models/final_model.pkl']

# Models Evaluations

In [84]:
import joblib
model_file1 = '../models/model_V100_7MM.pkl'

for model in list_pickles("../models/"):
    print("===========================")
    print(model)
    model = joblib.load(model)
    
    print("Evaluate on test data")
    results = model.predict([(inputboard), (inputmeta)])
    
    batch_size = 100  # Adjust the batch size according to your memory capacity
    num_samples = x_test.shape[0]
    
    mse_values = []
    mae_values = []
    for i in range(0, num_samples, batch_size):
        x_batch = results[i:i+batch_size]
        y_batch = y_test[i:i+batch_size]
        mse = tf.keras.losses.mean_squared_error(y_batch, x_batch).numpy()
        mae = tf.keras.losses.mean_absolute_error(y_batch, x_batch).numpy()
        mse_values.append(mse)
        mae_values.append(mae)
    
    overall_mse = np.mean(mse_values)
    overall_mae = np.mean(mae_values)
    print("Overall Mean Squared Error:", overall_mse)
    print("Overall Mean Absolute Error:", overall_mae)
    print("===========================")


../models/model_7MM_DL2.pkl
Evaluate on test data
31250/31250 [==============================] - 374s 12ms/step
Overall Mean Squared Error: 23675.867
Overall Mean Absolute Error: 54.089268
../models/model1.pkl
Evaluate on test data
31250/31250 [==============================] - 357s 11ms/step
Overall Mean Squared Error: 21682.86
Overall Mean Absolute Error: 50.157383
../models/model.pkl
Evaluate on test data
31250/31250 [==============================] - 364s 12ms/step
Overall Mean Squared Error: 22172.072
Overall Mean Absolute Error: 51.049458
../models/model_7MM_DL1.pkl
Evaluate on test data
31250/31250 [==============================] - 390s 12ms/step
Overall Mean Squared Error: 23491.162
Overall Mean Absolute Error: 53.369904
../models/model_7MM_DL0.pkl
Evaluate on test data
31250/31250 [==============================] - 411s 13ms/step
Overall Mean Squared Error: 23619.287
Overall Mean Absolute Error: 53.79201
../models/model_V100_7MM.pkl
Evaluate on test data
31250/31250 [========